In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from functools import lru_cache
import scipy.linalg as la

sns.set_context('notebook', font_scale=1.5)

In [2]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

**1**. (25 points)

In this exercise, we will practice using Pandas dataframes to explore and summarize a data set `heart`.

This data contains the survival time after receiving a heart transplant, the age of the patient and whether or not the survival time was censored

- Number of Observations - 69
- Number of Variables - 3

Variable name definitions::

- survival - Days after surgery until death
- censors - indicates if an observation is censored. 1 is uncensored
- age - age at the time of surgery

Answer the following questions (5 points each) with respect to the `heart` data set:

- How many patients were censored?
- What is the correlation coefficient between age and survival for uncensored patients? 
- What is the average age for censored and uncensored patients?
- What is the average survival time for censored and uncensored patients under the age of 45?
- What is the survival time of the youngest and oldest uncensored patient?


In [3]:
import statsmodels.api as sm
heart = sm.datasets.heart.load_pandas().data
heart.head(n=6)

,survival,censors,age
0,15.0,1.0,54.3
1,3.0,1.0,40.4
2,624.0,1.0,51.0
3,46.0,1.0,42.5
4,127.0,1.0,48.0
5,64.0,1.0,54.6


In [4]:
# Split df to make subsequent parts simpler
cen = heart[heart["censors"]==1]
unc = heart[heart["censors"]==0]

In [5]:
# Number of censored patients
cen.shape[0]

45

In [6]:
# Correlation between age and survival for uncensored patients
np.corrcoef(unc["age"], unc["survival"])[0,1]

0.09413217367925551

In [7]:
# Average age for censored/uncensored patients
heart.groupby("censors").mean()

,survival,age
censors,,
0.0,664.625000,41.729167
1.0,223.288889,48.484444


In [8]:
# Average survival time for patients under 45, censored/uncensored
heart[heart["age"]<45].groupby("censors").mean()

,survival,age
censors,,
0.0,712.818182,33.190909
1.0,169.909091,38.418182


In [9]:
# Survival time for youngest censored patient
unc[unc["age"] == min(unc["age"])]

,survival,censors,age
66,110.0,0.0,23.7


In [10]:
# Survival time for oldest censored patient
unc[unc["age"] == max(unc["age"])]

,survival,censors,age
58,339.0,0.0,54.4


**2**. (35 points)

- Consider a sequence of $n$ Bernoulli trials with success probabilty $p$ per trial. A string of consecutive successes is known as a success *run*. Write a function that returns the counts for runs of length $k$ for each $k$ observed in a dictionary. (10 points)

For example: if the trials were [0, 1, 0, 1, 1, 0, 0, 0, 0, 1], the function should return 
```
{1: 2, 2: 1})
```

Test that it does so.

- What is the probability of observing at least one run of length 5 or more when $n=100$ and $p=0.5$?. Estimate this from 100,000 simulated experiments. Is this more, less or equally likely than finding runs of length 7 or more when $p=0.7$? (10 points)

- There is an exact solution

$$
s_n = \sum_{i=1}^n{f_i} \\
f_n = u_n - \sum_{i=1}^{n-1} {f_i u_{n-i}} \\
u_n = p^k - \sum_{i=1}^{k-1} u_{n-i} p_i
$$

Implement the exact solution using caching to avoid re-calculations and calculate the same two probabilities found by simulation. (15 points)

In [11]:
# Write a function that returns the counts for runs of length k for each k observed in a dictionary
def run_counts(x):
    counts = {}
    streak = 0
    for i in x:
        if i==1:
            streak += 1
        else:
            if streak:
                if streak in counts.keys():
                    counts[streak] += 1
                else:
                    counts[streak] = 1
            streak = 0
    if streak:
        if streak in counts.keys():
            counts[streak] += 1
        else:
            counts[streak] = 1
    return counts

In [12]:
# Test
run_counts([0, 1, 0, 1, 1, 0, 0, 0, 0, 1])

{1: 2, 2: 1}

In [13]:
n = 10**2 # Trials
k = 10**5 # Experiments
p = 0.5   # Probability
s = 5     # Streak

In [14]:
# Helper function
def sim(n, k, p, s):
    xs = np.random.choice([0,1], (k, n), p = (1-p, p))
    return sum(max(d.keys()) >= s for d in map(run_counts, xs))/k

In [15]:
# First estimate
sim(n, k, p, s)

0.81044

In [16]:
# Second estimate
sim(n, k, 0.7, 7)

0.94931

In [17]:
@lru_cache()
def s(n, k, p):
    return sum(f(i, k, p) for i in range(1, n+1))

In [18]:
@lru_cache()
def f(n, k, p): 
    return u(n, k, p) - sum(f(i, k, p) * u(n-i, k, p) for i in range(1, n))

In [19]:
@lru_cache()
# Zero base case, all others defined recursively
def u(n, k, p):
    if n < k: 
        return 0
    return p**k - sum(u(n-i, k, p) * p**i for i in range(1, k))

In [20]:
# First solution
s(100, 5, 0.5)

0.8101095991963579

In [21]:
# Second solution
s(100, 7, 0.7)

0.9491817984156692

**3**. (40 points)

Given matrix $M$
```python
      [[7, 8, 8],
       [1, 3, 8],
       [9, 2, 1]]
```

- Normalize the given matrix $M$ so that all rows sum to 1.0.  (5 points)
- The normalized matrix can then be considered as a transition matrix $P$ for a Markov chain. Find the stationary distribution of this matrix in the following ways using `numpy` and `numpy.linalg` (or `scipy.linalg`):
    - By repeated matrix multiplication of a random probability vector $v$ (a row vector normalized to sum to 1.0) with $P$ using matrix multiplication with `np.dot`. (5 points)
    - By raising the matrix $P$ to some large power until it doesn't change with higher powers (see `np.linalg.matrix_power`) and then calculating $vP$ (10 points)
    - From the equation for stationarity $wP = w$, we can see that $w$ must be a left eigenvector of $P$ with eigenvalue $1$ (Note: np.linalg.eig returns the right eigenvectors, but the left eigenvector of a matrix is the right eigenvector of the transposed matrix). Use this to find $w$ using `np.linalg.eig`. (20 points)

Suppose $w = (w_1, w_2, w_3)$. Then from $wP = w$, we have:
\begin{align}
w_1 P_{11} + w_2 P_{21} + w_3 P_{31} &= w_1 \\
w_1 P_{12} + w_2 P_{22} + w_3 P_{32} &= w_2 \\
w_1 P_{13} + w_2 P_{23} + w_3 P_{331} &= w_3 \\
\end{align}
This is a singular system, but we also know that $w_1 + w_2 + w_3 = 1$. Use these facts to set up a linear system of equations that can be solved with `np.linalg.solve` to find $w$.

# Setup

In [22]:
# Set up matrix
xs = np.array([[7, 8, 8],
               [1, 3, 8],
               [9, 2, 1]])

In [23]:
# Normalize
xs = xs / np.sum(xs, axis=1).reshape(-1, 1)

In [24]:
# Verify everything worked
np.sum(xs, axis=1)

array([1., 1., 1.])

# Method 1

In [25]:
# Random vector
vec = np.random.random(3)
vec = vec / vec.sum()
vec

array([0.03185937, 0.10692264, 0.86121799])

In [26]:
# Multiply vec by xs a bunch of times
v1 = vec.copy()
for i in range(100):
    v1 = v1 @ xs

In [27]:
# Multiply once more, check for similary
vec_final = v1 @ xs
v1 == vec_final

array([ True,  True,  True])

In [28]:
# Stationary distribution
vec_final

array([0.39862184, 0.2605972 , 0.34078096])

# Method 2

In [29]:
# Raise xs to a high power
xs_1 = np.linalg.matrix_power(xs, 100)

In [30]:
# Multiply once more, check for similarity
xs_2 = xs_1 @ xs_1
np.testing.assert_allclose(xs_1, xs_2)

In [31]:
# Stationary distribution
vec @ xs_2

array([0.39862184, 0.2605972 , 0.34078096])

# Method 3

- Note: regularly $Av = \lambda v$
- Stationarity requires $wP = w$, so we need to find the *LEFT* eigenvector with an eigenvalue of 1
- To find left eigenvectors/values, must use transpose of $P$

In [32]:
# Find left eigenvalues/vectors
vals, vecs = np.linalg.eig(xs.T)

In [33]:
# Locate index of 1, corresponding eigenvector as row vector
idx = np.argmin(np.abs(vals-1))
v = np.real(vecs[:, idx]).T

In [34]:
# Stationary distribution
v/np.sum(v)

array([0.39862184, 0.2605972 , 0.34078096])

# Solving the system

In [35]:
# Subtract identity, substitute weight sum equation into row 3 to make matrix non-singular
ys = xs.copy()
ys = ys.T - np.eye(3)
ys[2, :] = 1
ys

array([[-0.69565217,  0.08333333,  0.75      ],
       [ 0.34782609, -0.75      ,  0.16666667],
       [ 1.        ,  1.        ,  1.        ]])

In [36]:
# Stationary distribution
np.linalg.solve(ys, [0,0,1])

array([0.39862184, 0.2605972 , 0.34078096])